![title](../logo.png)

# WMC-03: MODELAGEM MATEMÁTICA E COMPUTACIONAL NA COVID-19
# Proponente: UFJF
## Ministrantes: Rodrigo Weber dos Santos (UFJF), Marcelo Lobosco (UFJF) e Bernardo Martins Rocha (UFJF)

# Introdução

Conteúdo dessa aula:

01. Conceitos preliminares (Python, Jupyter-Notebook, Bibliotecas)
02. Ajuste de curva
03. Modelo matemático de COVID-19
04. Simulação dos modelos baseados em EDOs
05. Ajuste de parâmetros dos modelos
06. Propagação de incertezas
07. Análise de sensibilidade

## Alguns links importantes

- Python: https://www.python.org/
- NumPy: https://numpy.org/
- Lmfit: https://lmfit.github.io/lmfit-py/
- ChaosPy: https://chaospy.readthedocs.io/en/master/
- SALib: https://salib.readthedocs.io/en/latest/

# Imports

In [ ]:
import sys
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline 
!pip install mpld3
import mpld3
mpld3.enable_notebook()

from scipy.integrate import odeint
!pip install lmfit
import lmfit
from lmfit.lineshapes import gaussian, lorentzian
!pip install numdifftools

# UQ e SA
!pip install chaospy
!pip install SAlib

import warnings
warnings.filterwarnings('ignore')

Defaulting to user installation because normal site-packages is not writeable


# Exemplo de Ajuste de Curva

Considere um exemplo onde se deseja ajustar a seguinte curva (distribuição normal com algum ruído):

In [ ]:
np.random.seed(123)
x = np.linspace(0, 20.0, 1001)

# dist normal com ruido
media = 6.1
dp = 1.2
data = (gaussian(x, 21, media, dp) + np.random.normal(scale=0.1, size=x.size))  
plt.plot(x, data);

Então, definimos uma função que recebe x como primeiro argumento, e os parâmetros a serem ajustados (a, b, c)

In [ ]:
def f(x, a, b, c):
    return gaussian(x, a, b, c)

In [ ]:
mod = lmfit.Model(f)

# parametros (e algumas aproximacoes iniciais)
mod.set_param_hint("a", value=10.0, vary=True)
mod.set_param_hint("b", value=10.0, vary=True)
mod.set_param_hint("c", value=10.0, vary=True)

params = mod.make_params()

In [ ]:
result = mod.fit(data, params, method="leastsq", x=x)  # fitting

In [ ]:
plt.figure(figsize=(8,4))
result.plot_fit(datafmt="-");
result.best_values

In [ ]:
result

# Simulação de modelos matemáticos de COVID-19

Como visto nas outras aulas, alguns modelos de COVID-19 podem ser descritos como sistemas de equações diferenciais ordinárias (EDOs). O modelo mais simples para transmissão de doenças infecciosas é o modelo chamado SIR (Susceptible, Infectious, Recovered). Vamos apresentar as ideias com base nesse modelo.

## Modelo SIR (Susceptible, Infectious, or Recovered) 

$$
\begin{align}
\frac{d S}{d t} &=-\frac{\beta I S}{N} \\
\frac{d I}{d t} &=\frac{\beta I S}{N}-\gamma I \\
\frac{d R}{d t} &=\gamma I
\end{align}
$$

onde
- $S$: suscetíveis;
- $I$: infectados;
- $R$: recuperados;
- $N$: tamanho da população;
- $\beta$: taxa de transmissão;
- $\gamma$: taxa de recuperação.

## Simulação do modelo SIR

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.integrate as sig
import seaborn as sns
from scipy.integrate import odeint
#sns.set()

def sir(xs, t, ps):
    """
    SIR model
    """
    try:
        N = ps['n'].value
        beta = ps['beta'].value
        gama = ps['gama'].value
    except:
        N, beta, gama = ps
    # equations
    S,I,R = xs
    dSdt = - beta*S*I/N
    dIdt = beta*S*I/N - gama*I
    dRdt = gama*I
    return np.array([dSdt, dIdt, dRdt])

def plot_sir(x, y, labels):
    plt.plot(x, y[:,0], lw=2, label=labels[0])
    plt.plot(x, y[:,1], lw=2, label=labels[1])
    plt.plot(x, y[:,2], lw=2, label=labels[2])
    plt.xlabel('tempo')
    plt.ylabel('populacao')
    plt.legend(loc='best')
    plt.show()

In [ ]:
# parametros
N = 1000
beta = 0.2
gama = 0.1
params = (N,beta,gama)

# condicao inicial
i0, r0 = 1, 0
s0 = N-i0-r0

tf = 150
t = np.linspace(0,tf,150)
ts = (0, tf)
labels = ['suscetivel','infectado','recuperado']

# resolve EDOs
ic = np.array([s0,i0,r0])
X = odeint(sir, ic, t, args=(params,))

plot_sir(t, X/N, labels)

## Geração de dados sintéticos para exemplo de ajuste do modelo

In [ ]:
def model(t, x0, ps):
    """
    Solution to the ODE x'(t) = f(t,x,k) 
    with initial condition x(0) = x0
    """
    x = odeint(sir, x0, t, args=(ps,))
    return x

In [ ]:
np.set_printoptions(threshold=10)

# parametros
true_params = np.array((N,beta,gama))

# condicao inicial
i0, r0 = 1, 0
s0 = N-i0-r0
ic = (s0,i0,r0)
tf = 150
t = np.linspace(0,tf,50)

# gera dados
data = model(t, ic, true_params)
print("  S\t\t I\t\t R")
print(data)

# adiciona ruido de media=0 e dp=10
data += np.random.normal(loc=0.0, scale=10.0, size=data.shape)

# plot dos dados
plt.plot(t, data[:,0], 'o', label='suscetivel')
plt.plot(t, data[:,1], 'v', label='infectado')
plt.plot(t, data[:,2], '^', label='recuperado')
plt.xlabel('tempo')
plt.ylabel('populacao')
plt.legend(loc='best')
plt.title("dados sintéticos")
plt.show()

## Ajuste de parâmetros do modelo SIR

In [ ]:
def sir(xs, t, N, ps):
    """
    SIR model
    """
    try:
        beta = ps['beta'].value
        gama = ps['gama'].value
    except:
        beta, gama = ps
    # equations
    S,I,R = xs
    dSdt = - beta*S*I/N
    dIdt = beta*S*I/N - gama*I
    dRdt = gama*I
    return np.array([dSdt, dIdt, dRdt])

def model(t, x0, N, ps):
    """
    Solution to the ODE x'(t) = f(t,x,k) 
    with initial condition x(0) = x0
    """
    x = odeint(sir, x0, t, args=(N,ps))
    return x

def residual(ps, ts, ydata, N, ics):
    """
    Computes residual for NL least squares
    """
    ymodel = model(ts, ics, N, ps)
    return (ymodel - ydata).ravel()

def residual_i(ps, ts, ydata, N, ics):
    """
    Computes residual for NL least squares - infected only
    """
    ymodel = model(ts, ics, N, ps)
    return (ymodel[:,1] - ydata[:,1]) #.ravel()

Obs: O que faz essa função ravel() do Python/Numpy?

In [ ]:
xx = np.array([[1, 2, 3], [4, 5, 6]])
print(np.shape(xx))
np.ravel(xx)

In [ ]:
from lmfit import minimize, Parameters, Parameter, report_fit

# =================================================================
# Caso 1 - Sintetico
# =================================================================

# parametros
N = 1000
beta = 0.2
gama = 0.1 
true_params = np.array((beta,gama))

# condicao inicial
i0, r0 = 1, 0
s0 = N-i0-r0
ic = (s0,i0,r0)

# tempo
tf = 150
t = np.linspace(0,tf,50)

# gera dados
data = model(t, ic, N, true_params)

# adiciona ruido de media=0 e dp=20
data += np.random.normal(loc=0.0, scale=10.0, size=data.shape)

# =================================================================
# Fitting
# =================================================================

# initial guess
params = Parameters()
params.add('beta', value=0.5, min=0, max=10)
params.add('gama', value=0.5, min=0, max=10)

# fitting
result = minimize(residual, params, args=(t, data, N, ic), method='leastsq')

#result = minimize(residual_i, params, args=(t, data, N, ic), method='leastsq')

# display fitted statistics
report_fit(result)

# plot fitted model
labels = ['suscetivel','infectado','recuperado']
final = model(t, ic, N, result.params)

In [ ]:
# =================================================================
# Resultados
# =================================================================

#result
result.params


In [ ]:
# plots
plt.plot(t, data[:,0], 'o', label=labels[0] + ' (dados)')
plt.plot(t, data[:,1], 'v', label=labels[1] + ' (dados)')
plt.plot(t, data[:,2], '^', label=labels[2] + ' (dados)')
plt.plot(t, final[:,0], '-', lw=4, label=labels[0] + ' (modelo)')
plt.plot(t, final[:,1], '-', lw=4, label=labels[1] + ' (modelo)')
plt.plot(t, final[:,2], '-', lw=4, label=labels[2] + ' (modelo)')
plt.xlabel('tempo')
plt.ylabel('populacao')
plt.legend(loc='best')
plt.show()

## Experimentos com o ajuste

- Aumentar o ruído/erro/incerteza nos dados
- Variar os valores iniciais dos parâmetros
- Variar o método utilizado (LM, DE, etc)
- Ajuste de parâmetros do modelo, seguido de previsão pelo modelo
- Ajuste de parâmetros do modelo utilizando dados reais (Notebook separado)

## Ajuste e previsão

In [ ]:
# =================================================================
# Caso 2 - Sintetico - Ajuste + Previsao
# =================================================================

# parametros
N = 1000
beta = 0.2
gama = 0.1
true_params = np.array((beta,gama))

# condicao inicial
i0, r0 = 1, 0
s0 = N-i0-r0
ic = (s0,i0,r0)

# tempo
tf = 40
t = np.linspace(0,tf,50)

# gera dados
data = model(t, ic, N, true_params)

# adiciona ruido de media=0 e dp=10
data += np.random.normal(loc=0.0, scale=5.0, size=data.shape)

# =================================================================
# Fitting
# =================================================================

# initial guess
params = Parameters()
params.add('beta', value=0.5, min=0, max=10)
params.add('gama', value=0.5, min=0, max=10)

# fitting
result = minimize(residual, params, args=(t, data, N, ic), method='leastsq') #differential_evolution')
#result = minimize(residual_i, params, args=(t, data, N, ic), method='leastsq')

# display fitted statistics
#result
result.params

In [ ]:
# previsao ate o tempo tf=100
tf = 100
tt = np.linspace(0,tf,200)

# avalia modelo ajustado
final_true = model(tt, ic, N, true_params)

# avalia modelo real/true
final_forecast = model(tt, ic, N, result.params)

# plot data + previsao + modelo real
plt.plot(t, data[:,1], 'v', label=labels[1] + ' (dados)')
plt.plot(tt, final_forecast[:,1], '-', lw=4, label=labels[1] + ' (modelo)')
plt.plot(tt, final_true[:,1], '-', lw=4, label=labels[1] + ' (modelo-true)')
plt.xlabel('tempo')
plt.ylabel('populacao')
plt.legend(loc='best')
plt.show()

# Propagação de Incertezas

Existem vários métodos que podem ser utilizados para realizar a propagação de incertezas dos parâmetros do modelo, porém o método mais simples e utilizado é o método de Monte Carlo (MC). Vamos ilustrar o seu uso nesse contexto de forma prática e simplificada.

Vamos considerar, por exemplo, que os parâmetros encontrados no ajuste possuam ambas incertezas de cerva de 5% (+/-). Isso não é necessariamente verdade, mas, para simplificar a apresentação, vamos considerar estes valores. 

Também iremos supor que os parâmetros sejam governados por distribuições uniformes. Sendo assim, temos:
- $\beta \sim U(0.19, 0.21)$
- $\gamma \sim U(0.095, 0.105)$

A ideia básica do método de MC é gerar amostras dos parâmetros e simular o modelo (forward model) para cada valor dessas amostras e no final extrair as estatísticas e informações de interesse a partir dos dados gerados.

Vamos ver como isso funciona através de um exemplo com o modelo SIR.

In [ ]:
import chaospy as cp

# parametros
N = 1000
beta = 0.2
gama = 0.1

# condicao inicial
i0, r0 = 1, 0
s0 = N-i0-r0

tf = 150
size = 500
t = np.linspace(0,tf,size)
labels = ['suscetivel','infectado','recuperado']

# assumindo erro de 5% nos parametros
perc_beta = 0.01
perc_gama = 0.20

# distribuicao dos parametros
beta_l = beta - beta * perc_beta
beta_r = beta + beta * perc_beta
d_beta = cp.Uniform(beta_l, beta_r)

gama_l = gama - gama * perc_gama
gama_r = gama + gama * perc_gama
d_gama = cp.Uniform(gama_l, gama_r)

# distribuicao conjunta
dist = cp.J(d_beta, d_gama)
#print(dist)

# numero de amostras
nsamples = 100

# create vectors to contain the expectations and variances
samples = dist.sample(size=nsamples)
print(samples.T)
print(np.shape(samples.T))

In [ ]:
# solutions
sols_s = np.zeros((nsamples, size))
sols_i = np.zeros((nsamples, size))
sols_r = np.zeros((nsamples, size))
# mean values
exp_s = np.zeros(size)
exp_i = np.zeros(size)
exp_r = np.zeros(size)
# variance
var_s = np.zeros(size)
var_i = np.zeros(size)
var_r = np.zeros(size)

#
# Monte Carlo sampling 
# for each input sample
# compute the underlying solution)
#
for j, sample in enumerate(samples.T):
    #print(j,sample)
    
    # solve for this sample
    ic = np.array([s0,i0,r0])
    sol = model(t, ic, N, sample)
    #plot_sir(t, sol, labels)

    # store solution
    sols_s[j,:] = sol[:,0]
    sols_i[j,:] = sol[:,1]
    sols_r[j,:] = sol[:,2]

#    
# Compute statistics of the outputs at each time
#
for j in range(size):
    ss = sols_s[:,j]
    si = sols_i[:,j]
    sr = sols_r[:,j]
    exp_s[j] = np.mean(ss)
    exp_i[j] = np.mean(si)
    exp_r[j] = np.mean(sr)    
    var_s[j] = np.std(ss)   
    var_i[j] = np.std(si)   
    var_r[j] = np.std(sr)

# plot S
plt.plot(t, exp_s, lw=2, color='blue', label='S (mean)')
plt.fill_between(t, exp_s+var_s, exp_s-var_s, facecolor='blue', alpha=0.5, label='S (std)')
# plot I
plt.plot(t, exp_i, lw=2, color='red', label='I (mean)')
plt.fill_between(t, exp_i[:]+var_i[:], exp_i[:]-var_i, facecolor='red', alpha=0.5, label='I (std)')
# plot R
plt.plot(t, exp_r, lw=2, color='green', label='R (mean)')
plt.fill_between(t, exp_r+var_r, exp_r-var_r, facecolor='green', alpha=0.5, label='R (std)')
# plot settings
plt.xlabel("tempo")
plt.ylabel("numero de individuos")
plt.legend(loc='best')
plt.show()

# Análise de Sensibilidade

Análise de Sensibilidade baseada nos índices de Sobol.

O índice de sensibilidade de um parâmetro quantifica o seu impacto na incerteza da saída
Medem parte da variância da saída que podem ser atribuídas à variabilidade do parâmetro de entrada.

Índice de primeira-ordem (ou principal) e índice total.

Índice de Sobol de primeira-ordem $S_i$:

\begin{equation}
S_{i}=\frac{V\left[E\left(Y \mid X_{i}\right)\right]}{V(Y)}
\end{equation}

Propriedades: $S_i \in [0,1]$ e $\sum_{i} S_i \le 1$.

Índice de Sobol total $S_{T_i}$ (mede variações devido a interações de $x_i$ e outros parâmetros): 

\begin{equation}
S_{T_{i}}=\frac{E\left[V\left(Y \mid \mathbf{X}_{\sim i}\right)\right]}{V(Y)}=1-\frac{V\left[E\left(Y \mid \mathbf{X}_{\sim i}\right)\right]}{V(Y)}
\end{equation}

Veremos a seguir um exemplo de como se calcular estes índices de sensibilidade para o modelo SIR utilizando a biblioteca SAlib.

Para simplificar o tratamento, vamos considerar Y como um ponto da curva de S,I ou R.

In [ ]:
from SALib.sample import saltelli
from SALib.analyze import sobol

In [ ]:
N = 1000
problem = {
    "num_vars": 2,
    "names": [ "beta", "gama" ],
    "bounds": [ [0.19, 0.21], [0.095, 0.105] ]
}

# gera as amostras de entrada dos parametros
param_vals = saltelli.sample(problem, N, calc_second_order=True)
print(np.shape(param_vals))
print(param_vals)

Observe que param_vals é uma matriz do NumPy cuja dimensão é 60000 by 2 para o caso com N=10000. O gerador de amostras de Saltelli gerou 60000 amostras.

O gerador de amostras de Saltelli gera N∗(2D+2) amostras, onde N é o número de 10000 é o argumento escolhido (para este exemplo) e D é 2 neste caso (o número de parâmetros de entrada). 

O argumento calc_second_order=False não irá calcular os índices de segunda ordem de Sobol (índice total), o que faz com que a matriz de amostras tenha apenas N∗(D+2) linhas.

In [ ]:
# array para armazenar uma quantidade de interesse
# a ser avaliada pela analise de sensibilidade
Ns = param_vals.shape[0]
Y = np.empty([Ns])
print(np.shape(Y))

In [ ]:
# configuracao do problema SIR
# parametros
Np = 1000

# condicao inicial
i0, r0 = 1, 0
s0 = Np-i0-r0

# tempo
tf = 150
size = 150
t = np.linspace(0,tf,size)

# avalia o modelo (SIR) para cada parâmetro de entrada
for i in range(Ns):
    x = param_vals[i]
    
    #print(f"amostra {i}: ", x)
    
    # resolve o problema SIR
    ic = np.array([s0,i0,r0])
    sol = model(t, ic, Np, x)
    
    # sol é uma matriz (npassos x 3)
    # 0-> suscetivel
    # 1-> infectado
    # 2-> recuperado
    
    # extrai a quantidade de interesse definida
    
    # num infectados no tempo t=60
    Y[i] = sol[60, 1]
    
    # num de recuperados no tempo t=60
    #Y[i] = sol[60, 2]
    
# fim do loop   

print("Amostras calculadas com sucesso")

In [ ]:
print("Calculando indices de Sobol\n")

# estimate the sensitivity indices using Sobol's method
sensitivity = sobol.analyze(problem, Y, calc_second_order=True)

# firstorder indices
print("Indice de Sobol princial or de primeira ordem")
print(sensitivity['S1'])
print()

# higher-order indices
print("Indice de Sobol total ou de alta ordem")
print(sensitivity['ST'])

## Cálculo dos índices de Sobol para todos instantes de tempo

In [ ]:
# array para armazenar uma quantidade de interesse
# a ser avaliada pela analise de sensibilidade
Ns = param_vals.shape[0]
Ys = np.empty([Ns,size])
Yi = np.empty([Ns,size])
Yr = np.empty([Ns,size])

# avalia o modelo (SIR) para cada parâmetro de entrada
for i in range(Ns):
    x = param_vals[i]
    
    print(f"amostra {i}: ", x)
    
    # resolve o problema SIR
    ic = np.array([s0,i0,r0])
    sol = model(t, ic, Np, x)
    
    # sol é uma matriz (npassos x 3)
    # 0-> suscetivel
    # 1-> infectado
    # 2-> recuperado
    
    # extrai a quantidade de interesse definida
    Ys[i,:] = sol[:,0]
    Yi[i,:] = sol[:,1]
    Yr[i,:] = sol[:,2]    
    
# fim do loop   

print("Amostras calculadas com sucesso")

In [ ]:
print("Calculando indices de Sobol\n")

Ss = np.empty([2,size])
Si = np.empty([2,size])
Sr = np.empty([2,size])

for j in range(size):
   
    # estimate the sensitivity indices using Sobol's method
    sens_s = sobol.analyze(problem, Ys[:,j], calc_second_order=True)
    sens_i = sobol.analyze(problem, Yi[:,j], calc_second_order=True)
    sens_r = sobol.analyze(problem, Yr[:,j], calc_second_order=True)

    Ss[:,j] = sens_s['S1']
    Si[:,j] = sens_i['S1']
    Sr[:,j] = sens_r['S1']
    #print(j)

print("OK")

# plot sobol indices for Infectados
plt.plot(t, Si[0,:], lw=2, color='blue', label='beta - Sobol main')
plt.plot(t, Si[1,:], lw=2, color='red', label='gama - Sobol main')

# plot settings
plt.xlabel("tempo")
plt.ylabel("indice de Sobol")
plt.legend(loc='best')
plt.show()

# Algumas observações

- Tanto na propagação de incertezas (UQ direta) quanto na análise de sensibilidade (SA) foi preciso escolher um intervalo para os parâmetros de entrada.

- Aqui nessa aula, essa escolha foi feita de forma simplificada.

- O ideal é obter essas informações após o ajuste dos parâmetros (lmfit fornece algumas estimativas).

- Uma abordagem mais apropriada é utilizar métodos de quantificação de incertezas inversa (UQ inversa), a qual fornece uma distribuição de probabilidade para cada parâmetro. Quanto maior a dispersão dessa distribuição para um determinado parâmetro, maior a incerteza deste parâmetro. Ex: Markov-Chain Monte-Carlo.

# Contatos

Email
- bernardomartinsrocha@ice.ufjf.br

Repositório no GitHub:
- https://github.com/rochabm/curso_sbpc_2021

# Referências

- Livro "An Introduction to Infectious Disease Modelling". Autores: Emilia Vynnycky, Richard White
- Livro "Uncertainty Quantification and Predictive Computational Science: A Foundation for Physical Scientists and Engineers". Autor: Ryan G. McClarren
- Livro "Global Sensitivity Analysis. The Primer - Andrea Saltelli" (Obs: disponível para download no site do autor)
- Site "Towards Data Science"
- https://towardsdatascience.com/infectious-disease-modelling-fit-your-model-to-coronavirus-data-2568e672dbc7
- https://medium.com/analytics-vidhya/coronavirus-in-italy-ode-model-an-parameter-optimization-forecast-with-python-c1769cf7a511